# InPars - Finetuning

Author: Monique Monteiro (moniquelouise@gmail.com)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
main_dir = "/content/gdrive/MyDrive/Unicamp-aula-9"

## Libraries installation

In [ ]:
!pip install transformers -q

In [ ]:
!pip install jsonlines -q

In [123]:
!pip install evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.7 MB/s eta 0:00:00


In [124]:
!pip install trectools -q

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
import random
import torch
import torch.nn.functional as F
import numpy as np

Random seeds definition, to enable replication of results.

In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

## Dataset processing

In [ ]:
dataset_file = f"{main_dir}/trec-covid/monique_monteiro_1000_queries.jsonl"

In [ ]:
!head {dataset_file}

{"query": "What is the most suitable protein for a diagnostic approach for Salmonella Enteritidis and why?", "positive_doc_id": "m1cmkkw3", "negative_doc_ids": ["0o3mryu1", "qpq7i1ya", "j5mkparg", "auoo0dm5", "dqfvrerw"]}
{"query": "What is Cryptosporidium parvum and why is it a major cause of disease in both humans and animals?", "positive_doc_id": "ukbl0svm", "negative_doc_ids": ["k3u2nvpe", "gc11fyms", "xoq9qblv", "20o4ufa3", "3huo5nf0"]}
{"query": "What is the role of the renin-angiotensin-aldosterone system in the context of SARS-CoV-2 infection?", "positive_doc_id": "12o4zey2", "negative_doc_ids": ["6gd6nwpu", "dt4t2wos", "8zwfkken", "sv7xpi4f", "6pf73z08"]}
{"query": "What are the functions of individual endolysosomal proteases in cellular processes such as autophagy and lipoprotein particle degradation?", "positive_doc_id": "eqv6a7tj", "negative_doc_ids": ["gmrty2uu", "uzn214j6", "032utjfh", "efet3ozc", "0muwl6oc"]}
{"query": "What is the prevalence of olfactory dysfunction in 

In [ ]:
import jsonlines

id_to_text = dict()

with jsonlines.open(f"{main_dir}/trec-covid/corpus.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    text = item["title"] + ' ' + item["text"]
    id_to_text[id] = text

In [ ]:
import json

dataset = []
dataset_ids = []
i = 0

with open(dataset_file, 'r') as f:
    for line in f:
        data = json.loads(line)
        query = data["query"]
        positive_doc_id = data["positive_doc_id"]
        negative_doc_ids = data["negative_doc_ids"]
        
        #Chooses a random negative document
        negative_doc_id = random.choice(negative_doc_ids)

        #Gets the documents texts
        positive_doc = id_to_text[positive_doc_id]
        negative_doc = id_to_text[negative_doc_id]

        dataset.append((query, positive_doc, negative_doc))
        dataset_ids.append((i, positive_doc_id, negative_doc_id))

        i+=1


In [ ]:
import pandas as pd

df = pd.DataFrame(dataset, columns=['query', 'pos', 'neg'])

In [ ]:
df.head()

,query,pos,neg
0,What is the most suitable protein for a diagno...,Rapid identification of novel antigens of Salm...,DNA vaccines and their applications in veterin...
1,What is Cryptosporidium parvum and why is it a...,Cryptosporidium and host resistance: historica...,The social and political impact of animal dise...
2,What is the role of the renin-angiotensin-aldo...,Understanding the Renin-Angiotensin-Aldosteron...,Integrative Physiological Aspects of Brain RAS...
3,What are the functions of individual endolysos...,Specific functions of lysosomal proteases in e...,Inhibitory effect of antisense aminopeptidase ...
4,What is the prevalence of olfactory dysfunctio...,Olfactory and rhinological evaluations in SARS...,Pre-Procedural Surveillance Testing for SARS-C...


In [ ]:
df_pos = pd.DataFrame()
df_neg = pd.DataFrame()

for index, row in df.iterrows():
  df_pos = df_pos.append({"query":row[0], "passage":row[1], "score":1.0}, ignore_index=True)
  df_neg = df_neg.append({"query":row[0], "passage":row[2], "score":0.0}, ignore_index=True)

In [ ]:
df_pos.head()

,query,passage,score
0,What is the most suitable protein for a diagno...,Rapid identification of novel antigens of Salm...,1.0
1,What is Cryptosporidium parvum and why is it a...,Cryptosporidium and host resistance: historica...,1.0
2,What is the role of the renin-angiotensin-aldo...,Understanding the Renin-Angiotensin-Aldosteron...,1.0
3,What are the functions of individual endolysos...,Specific functions of lysosomal proteases in e...,1.0
4,What is the prevalence of olfactory dysfunctio...,Olfactory and rhinological evaluations in SARS...,1.0


In [ ]:
df_neg.head()

,query,passage,score
0,What is the most suitable protein for a diagno...,DNA vaccines and their applications in veterin...,0.0
1,What is Cryptosporidium parvum and why is it a...,The social and political impact of animal dise...,0.0
2,What is the role of the renin-angiotensin-aldo...,Integrative Physiological Aspects of Brain RAS...,0.0
3,What are the functions of individual endolysos...,Inhibitory effect of antisense aminopeptidase ...,0.0
4,What is the prevalence of olfactory dysfunctio...,Pre-Procedural Surveillance Testing for SARS-C...,0.0


In [ ]:
df_pos.dtypes

query       object
passage     object
score      float64
dtype: object

In [ ]:
from sklearn.model_selection import train_test_split

X_train_pos = df_pos.drop("score", axis=1)
Y_train_pos = df_pos["score"]

X_train_pos, X_val_pos, Y_train_pos, Y_val_pos = train_test_split(X_train_pos, Y_train_pos, test_size=0.1, random_state=42)

X_train_neg = df_neg.drop("score", axis=1)
Y_train_neg = df_neg["score"]

X_train_neg, X_val_neg, Y_train_neg, Y_val_neg = train_test_split(X_train_neg, Y_train_neg, test_size=0.1, random_state=42)
     

In [ ]:
X_train = pd.concat([X_train_pos, X_train_neg], axis=0, ignore_index=True)
Y_train = pd.concat([Y_train_pos, Y_train_neg], axis=0, ignore_index=True)
X_val = pd.concat([X_val_pos, X_val_neg], axis=0, ignore_index=True)
Y_val = pd.concat([Y_val_pos, Y_val_neg], axis=0, ignore_index=True)

## Finetuning

Data preparation, with tokenization and dataset/dataloaders construction.

In [ ]:
#model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"
model_name = 'microsoft/MiniLM-L12-H384-uncased'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from statistics import mean, stdev

lengths = [len(tokens) for tokens in tokenizer(list(X_train["passage"])[:1_000])['input_ids']]
print(f'Mean length in tokens: {mean(lengths):0.2f}')
print(f'Stdev length in tokens: {stdev(lengths):0.2f}')

Mean length in tokens: 321.51
Stdev length in tokens: 144.16


In [ ]:
max_length = 512
max_length_query = int(max_length/4)
max_length_passage = int(3*max_length/4)
max_length_query, max_length_passage

(128, 384)

In [ ]:
train_queries = list(X_train["query"])
train_passages = list(X_train["passage"])
val_queries = list(X_val["query"])
val_passages = list(X_val["passage"])

train_queries_tokenized = tokenizer(train_queries, truncation=True, max_length=max_length_query)
train_passages_tokenized = tokenizer(train_passages, truncation=True, max_length=max_length_passage)
val_queries_tokenized = tokenizer(val_queries, truncation=True, max_length=max_length_query)
val_passages_tokenized = tokenizer(val_passages, truncation=True, max_length=max_length_passage)

In [ ]:
from torch.utils import data

class Dataset(data.Dataset):
    def __init__(self, queries, passages, targets):
        self.queries = queries
        self.passages = passages
        self.targets = targets
    
    def __len__(self):
        return len(self.queries['input_ids'])
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.queries['input_ids'][idx] + self.passages['input_ids'][idx],
            'attention_mask': self.queries['attention_mask'][idx] + self.passages['attention_mask'][idx],
            #'labels': float(self.targets[idx])
            'labels': int(self.targets[idx])
        }


In [ ]:
dataset_train = Dataset(train_queries_tokenized, train_passages_tokenized, Y_train)
assert len(dataset_train[0]['input_ids']) > 0
assert len(dataset_train[1]['attention_mask']) > 0

In [ ]:
dataset_valid = Dataset(val_queries_tokenized, val_passages_tokenized, Y_val)

In [ ]:
from transformers import BatchEncoding

# Tokens do tipo "pad" para textos com tamanho inferior ao máximo suportado.
def collate_fn(batch):
  return BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))


In [ ]:
dataloader_train = data.DataLoader(dataset_train, batch_size=32, shuffle=True, collate_fn=collate_fn)
dataloader_valid = data.DataLoader(dataset_valid, batch_size=32, shuffle=False, collate_fn=collate_fn)

for batch in dataloader_train:
    assert batch['input_ids'].shape[0] <= dataloader_train.batch_size
    assert batch['input_ids'].shape[1] <= max_length
    break

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


## Training Loop

In [ ]:
from tqdm import tqdm 

def evaluate(model, dataloader, set_name):
    losses = []
    correct = 0
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False):
            outputs = model(**batch.to(device))
            loss_val = outputs.loss
            losses.append(loss_val.cpu().item())
            preds = outputs.logits.argmax(dim=1)
            correct += (preds == batch['labels']).sum().item()

    print(f'{set_name} loss: {mean(losses):0.3f}; {set_name} accuracy: {correct / len(dataloader.dataset):0.3f}')
     

In [ ]:
from torch import nn
from torch import optim
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def train(model_name, epochs = 5, lr=5e-5):
  model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
  print('Parameters', model.num_parameters())

  optimizer = optim.AdamW(model.parameters(), lr)
  num_training_steps = epochs * len(dataloader_train)

  num_warmup_steps = int(num_training_steps * 0.1)
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

  evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

  # Training loop
  for epoch in tqdm(range(epochs), desc='Epochs'):
      model.train()
      train_losses = []
      for batch in tqdm(dataloader_train, mininterval=0.5, desc='Train', disable=False):
          optimizer.zero_grad()
          outputs = model(**batch.to(device))
          loss = outputs.loss
          loss.backward()
          optimizer.step()
          scheduler.step()
          train_losses.append(loss.cpu().item())

      print(f'Epoch: {epoch + 1} Training loss: {mean(train_losses):0.2f}')
      evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')
  
  return model

In [ ]:
MODELS_PATH = '/content/gdrive/MyDrive/Unicamp-aula-9'

In [ ]:
model = train(model_name, 5)
model_name = model_name.replace('/','_')
model.save_pretrained(f'{MODELS_PATH}/models_ranker_{model_name}')
tokenizer.save_pretrained(f'{MODELS_PATH}/tokenizer_ranker')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters 33360770


Valid:   0%|          | 0/7 [00:00<?, ?it/s]

Valid loss: 0.697; Valid accuracy: 0.500


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

...

Train:   0%|          | 0/57 [00:00<?, ?it/s]

Epoch: 5 Training loss: 0.02


Valid:   0%|          | 0/7 [00:00<?, ?it/s]

Valid loss: 0.077; Valid accuracy: 0.980


('/content/gdrive/MyDrive/Unicamp-aula-9/tokenizer_ranker/tokenizer_config.json',
 '/content/gdrive/MyDrive/Unicamp-aula-9/tokenizer_ranker/special_tokens_map.json',
 '/content/gdrive/MyDrive/Unicamp-aula-9/tokenizer_ranker/vocab.txt',
 '/content/gdrive/MyDrive/Unicamp-aula-9/tokenizer_ranker/added_tokens.json',
 '/content/gdrive/MyDrive/Unicamp-aula-9/tokenizer_ranker/tokenizer.json')

## Reranking

Evaluation on TREC-COVID

In [ ]:
id_to_query = dict()

with jsonlines.open(f"{main_dir}/trec-covid/queries.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    text = item["text"]
    id_to_query[id] = text

In [ ]:
import jsonlines

id_to_doc = dict()

with jsonlines.open(f"{main_dir}/trec-covid/corpus.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    text = item["title"] + ' ' + item["text"]
    id_to_doc[id] = text

In [ ]:
!head {main_dir}/trec-covid/run.trec-covid.bm25tuned.txt

1 Q0 dv9m19yk 1 4.158100 Anserini
1 Q0 kgifmjvb 2 3.338900 Anserini
1 Q0 wmfcey6f 3 3.338899 Anserini
1 Q0 safr9z37 4 3.220100 Anserini
1 Q0 0paafp5j 5 3.207300 Anserini
1 Q0 96zsd27n 6 3.207299 Anserini
1 Q0 4dtk1kyh 7 3.184800 Anserini
1 Q0 lhd0jn0z 8 2.903200 Anserini
1 Q0 55dihml5 9 2.899800 Anserini
1 Q0 qtx0d5f8 10 2.888800 Anserini


In [ ]:
import pickle
import os

tokenized_queries = None
tokenized_passages = None

if os.path.exists(f"{main_dir}/trec-covid/tok_queries_test.pickle"):
  with open(f"{main_dir}/trec-covid/tok_queries_test.pickle", "rb") as f:
    print("Loading test queries...")
    tokenized_queries = pickle.load(f) 

if os.path.exists(f"{main_dir}/trec-covid/tok_passages_test.pickle"):
  with open(f"{main_dir}/trec-covid/tok_passages_test.pickle", "rb") as f:
    print("Loading test passages...")
    tokenized_passages = pickle.load(f) 

query_ids = []
queries = []
passage_ids = []
passages = []

with open(f'{main_dir}/trec-covid/run.trec-covid.bm25tuned.txt') as f:
  for line in f:
      fields = line.strip().split()
      query_id = fields[0]
      query_ids.append(query_id)
      passage_id = fields[2]
      passage_ids.append(passage_id)
      
      if not tokenized_queries:
        query_text = id_to_query[query_id]
        queries.append(query_text)

      if not tokenized_passages:
        passage_text = id_to_doc[passage_id]
        passages.append(passage_text)

if not tokenized_queries:
  tokenized_queries = tokenizer(queries, max_length=max_length_query, truncation=True)

  with open(f"{main_dir}/trec-covid/tok_queries_test.pickle", 'wb') as f:
    pickle.dump(tokenized_queries, f)

if not tokenized_passages:
  tokenized_passages = tokenizer(passages, max_length=max_length_passage, truncation=True)

  with open(f"{main_dir}/trec-covid/tok_passages_test.pickle", 'wb') as f:
    pickle.dump(tokenized_passages, f)

In [ ]:
dataset_test = Dataset(tokenized_queries, tokenized_passages, [1]*len(tokenized_queries['input_ids']))

In [ ]:
dataloader_test = data.DataLoader(dataset_test, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [ ]:

def evaluate_test_dataset(model, dataloader, set_name, use_logits=False):
    scores = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False):
            outputs = model(**batch.to(device))
            if use_logits:
              # Usa os logits brutos
              pos_score = outputs.logits[:,1]
            else:
              # Usa os logits normalizados pelo softmax (por default)
              pos_score = torch.softmax(outputs.logits,1)[:,1]
            scores = scores + pos_score.tolist()
    return scores

In [153]:
def evaluate_ndcg_10(scores, model_name, eval_desc):
  zipped_results = []

  #Por alguma razão misteriosa, o zip do Python não funcionou, deixou a lista 
  #vazia ou impossível de ser iterada.
  for i, query_id in enumerate(query_ids):
    zipped_results.append((query_id, passage_ids[i], scores[i]))

  #Quebra a lista em sublistas por query
  prev_query_id = -1
  sublists = []
  current_list = []

  for query_id, passage_id, score in zipped_results:
    if query_id != prev_query_id:
      if len(current_list) > 0:
        sublists.append(current_list)
        current_list = []
    current_list.append((query_id, passage_id, score))
    prev_query_id = query_id

  if len(current_list) > 0:
    sublists.append(current_list)

  # Ordena cada sublista
  sorted_list = []

  for sublist in sublists:
    sorted_sublist = sorted(sublist, key=lambda x: x[2], reverse=True)
    sorted_list += sorted_sublist

  # Gera o arquivo de run no formato TREC
  trec_run_file = f"{main_dir}/trec-covid/run.trec-covid.bert_reranked_{model_name}_{eval_desc}.trec"
  with open(trec_run_file, "w") as f:
    for i, (query_id, passage_id, score) in enumerate(sorted_list):
      f.write(f'{query_id}\t{passage_id}\t{i+1}\t{score}\tbert_reranked_{model_name}\n')

  return trec_run_file

In [121]:

softmax_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test')
logit_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test', use_logits=True)

Test:   0%|          | 0/1563 [00:00<?, ?it/s]

Test:   0%|          | 0/1563 [00:00<?, ?it/s]

In [154]:

trec_file_softmax = evaluate_ndcg_10(softmax_scores, model_name, "softmax")
trec_file_logits = evaluate_ndcg_10(logit_scores, model_name, "logits")

In [155]:
!head {trec_file_softmax}

1	ews32six	1	0.9927167296409607	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	k4l70wf6	2	0.9926959276199341	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	fraczoxu	3	0.9926958084106445	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	zpb9jicw	4	0.9926846027374268	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	dt0b87me	5	0.9926679730415344	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	8f5m0gej	6	0.9926672577857971	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	67fyu0i5	7	0.9926584362983704	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	o7kl258h	8	0.9926577210426331	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	1ew0p6x7	9	0.9926537275314331	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	2cwvga0k	10	0.9926504492759705	bert_reranked_microsoft_MiniLM-L12-H384-uncased


In [141]:
import pandas as pd

qrel = pd.read_csv(f"{main_dir}/trec-covid/test.tsv", sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rel"])
qrel["q0"] = "q0"
qrel = qrel.to_dict(orient="list")

In [142]:
!head {main_dir}/trec-covid/test.tsv

query-id	corpus-id	score
1	005b2j4b	2
1	00fmeepz	1
1	g7dhmyyo	2
1	0194oljo	1
1	021q9884	1
1	02f0opkr	1
1	047xpt2c	0
1	04ftw7k9	0
1	pl9ht0d0	0


In [143]:
from evaluate import load

def eval_ndcg10(run):
  trec_eval = load("trec_eval")
  results = trec_eval.compute(predictions=[run], references=[qrel])
  return results['NDCG@10'] 

In [156]:
import pandas as pd

run_trec_file_softmax = pd.read_csv(trec_file_softmax, sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rank", "score", "system"])
run_trec_file_softmax["q0"] = "q0"
run_trec_file_softmax = run_trec_file_softmax.to_dict(orient="list")

In [157]:
import pandas as pd

run_trec_file_logits = pd.read_csv(trec_file_logits, sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rank", "score", "system"])
run_trec_file_logits["q0"] = "q0"
run_trec_file_logits = run_trec_file_logits.to_dict(orient="list")

In [158]:
eval_ndcg10(run_trec_file_softmax)

0.6110263574494792

In [159]:
eval_ndcg10(run_trec_file_logits)

0.6125945845401235